# Лабораторная работа №6

## Задание 1 
###### Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [3]:
data = pd.read_csv('database.csv',low_memory=False)
data = data.dropna() # Заполнение пропущенных значений
subset_data = data.sample(n=11000, random_state=42)

###### Преобразование категориальных переменных в числовой формат

In [4]:
cat_features = ['Agency Name', 'Agency Type', 'City', 'State', 'Month', 'Crime Type', 
                'Victim Sex', 'Victim Race', 'Victim Ethnicity', 'Perpetrator Sex', 
                'Perpetrator Race', 'Perpetrator Ethnicity', 'Relationship', 'Weapon']
for cat in cat_features:
    subset_data[cat] = subset_data[cat].astype('category').cat.codes

###### Определение целевой переменной и признаков

In [5]:
X = subset_data.drop(columns=['Crime Solved', 'Record ID', 'Agency Code', 'Incident', 'Record Source'])
y = subset_data['Crime Solved'].astype('category').cat.codes

###### Разделение данных на обучающую и тестовую выборки

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Создание модели CatBoost
###### Перебор гиперпараметров не использовался, т.к. занимает очень много времени

In [7]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           loss_function='Logloss',
                           eval_metric='Accuracy',
                           random_seed=42)

###### Обучение модели

In [8]:
model.fit(X_train, y_train, cat_features=cat_features, eval_set=(X_test, y_test), verbose=False)

### Оценка модели

In [9]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.9995454545454545


###### Отчет о классификации

In [10]:
report = classification_report(y_test, y_pred, target_names=['Not Solved', 'Solved'])
print(report)

              precision    recall  f1-score   support

  Not Solved       1.00      1.00      1.00       650
      Solved       1.00      1.00      1.00      1550

    accuracy                           1.00      2200
   macro avg       1.00      1.00      1.00      2200
weighted avg       1.00      1.00      1.00      2200

